# Requirements

In [1]:
# !pip install facenet_pytorch

In [3]:
# !pip install opencv-python

In [5]:
# !pip install unrar

In [6]:
import torch
from torch import optim
from torch.utils.data import DataLoader, SubsetRandomSampler
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import numpy as np
import zipfile
import uuid
import time
import cv2
import os
import warnings
warnings.filterwarnings('ignore')

# Collect Data

In [ ]:
# root_dir = "/content/Train_Data"

# for i in range(300):
#   cap = cv2.VideoCapture(0)
#   print(f"Taking Picture number {i} ......")
#   time.sleep(5)
#   ret, frame = cap.read()
#   imgname = os.path.join(root_dir, f"_{i}.jpg")
#   cv2.imwrite(imgname, frame)
#   cv2.imshow('frame', frame)
#   time.sleep(3)

#   if cv2.waitKey(1) & 0xFF == ord('q'):
#     break

# cap.release()
# cv2.destroyAllWindows()

In [ ]:
# from IPython.display import display, Javascript
# from google.colab.output import eval_js
# from base64 import b64decode

# # def take_photo(filename='/content/Train_Data/photo.jpg', quality=0.8):
#   # root = "/content/Train_Data"
#   # filename = input("Image named as ")
#   # filename = os.path.join(root, filename + ".jpg")

# def take_photo(filename, quality=0.8):
#   js = Javascript('''
#     async function takePhoto(quality) {
#       const div = document.createElement('div');
#       const capture = document.createElement('button');
#       capture.textContent = 'Capture';
#       div.appendChild(capture);

#       const video = document.createElement('video');
#       video.style.display = 'block';
#       const stream = await navigator.mediaDevices.getUserMedia({video: true});

#       document.body.appendChild(div);
#       div.appendChild(video);
#       video.srcObject = stream;
#       await video.play();

#       // Resize the output to fit the video element.
#       google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

#       // Wait for Capture to be clicked.
#       await new Promise((resolve) => capture.onclick = resolve);

#       const canvas = document.createElement('canvas');
#       canvas.width = video.videoWidth;
#       canvas.height = video.videoHeight;
#       canvas.getContext('2d').drawImage(video, 0, 0);
#       stream.getVideoTracks()[0].stop();
#       div.remove();
#       return canvas.toDataURL('image/jpeg', quality);
#     }
#     ''')
#   display(js)
#   data = eval_js('takePhoto({})'.format(quality))
#   binary = b64decode(data.split(',')[1])
#   with open(filename, 'wb') as f:
#     f.write(binary)
#   return filename

In [ ]:
# from IPython.display import Image

# try:
#   for index in range(9):
#     print(f"Taking Picture number {index+100} ......")
#     root = "/content/Train_Data"
#     filename = os.path.join(root, "Sample_" + str(index) + ".jpg")
#     path = take_photo(filename)
#     print('Saved to {}'.format(path))
#     print("-"*40)
#     time.sleep(3)

# except Exception as err:
#   print(str(err))

Taking Picture number 100 ......


<IPython.core.display.Javascript object>

Saved to /content/Train_Data/Sample_0.jpg
----------------------------------------
Taking Picture number 101 ......


<IPython.core.display.Javascript object>

Saved to /content/Train_Data/Sample_1.jpg
----------------------------------------
Taking Picture number 102 ......


...

Saved to /content/Train_Data/Sample_6.jpg
----------------------------------------
Taking Picture number 107 ......


<IPython.core.display.Javascript object>

Saved to /content/Train_Data/Sample_7.jpg
----------------------------------------
Taking Picture number 108 ......


<IPython.core.display.Javascript object>

Saved to /content/Train_Data/Sample_8.jpg
----------------------------------------


# Fine-Tune

In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cpu


In [8]:
rar_file = '/content/data.rar'

# Create a directory to extract files to
extract_dir = '/content/'
os.makedirs(extract_dir, exist_ok=True)

# Extract the RAR file
!unrar x -r {rar_file} {extract_dir}


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/data.rar

Creating    /content/data                                             OK
Creating    /content/data/Person1                                     OK
Extracting  /content/data/Person1/Sample_0.jpg                             2%  OK 
Extracting  /content/data/Person1/Sample_1.jpg                             4%  OK 
Extracting  /content/data/Person1/Sample_10.jpg                            6%  OK 
Extracting  /content/data/Person1/Sample_11.jpg                            8%  OK 
Extracting  /content/data/Person1/Sample_12.jpg                            9%  OK 
Extracting  /content/data/Person1/Sample_13.jpg                           11%  OK 
Extracting  /content/data/Person1/Sample_14.jpg                           13%  OK 
Extracting  /content/data/Person1/Sample_15.jpg                           15%  OK 
Extract

In [9]:
len(os.listdir("/content/data/Person1"))

50

In [10]:
data_dir = '/content/data/'

batch_size = 16
epochs = 30
workers = 0 if os.name == 'nt' else 8

In [11]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

In [12]:
dataset = datasets.ImageFolder(data_dir, transform=transforms.Resize((512, 512)))
dataset.samples = [
    (p, p.replace(data_dir, data_dir + '_cropped'))
        for p, _ in dataset.samples
]

loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    collate_fn=training.collate_pil
)

for i, (x, y) in enumerate(loader):
    mtcnn(x, save_path=y)
    print('\rBatch {} of {}'.format(i + 1, len(loader)), end='')

# Remove mtcnn to reduce GPU memory usage
del mtcnn

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Batch 3 of 4

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Batch 4 of 4

In [19]:
# !rm -R /content/data/Person1
# !ls /content/data -a   #to make sure that the deletion has occurred

In [15]:
# !zip -r /content/dataset.zip /content/data

In [16]:
resnet = InceptionResnetV1(
    classify=True,
    pretrained='vggface2',
    num_classes=len(dataset.class_to_idx)
).to(device)

  0%|          | 0.00/107M [00:00<?, ?B/s]



> Error Handling:

A hidden .ipynb file causes error, so make sure to run the cells bellow with respect to the directory of your dataset



In [21]:
!rm -R /content/data/.ipynb_checkpoints

In [22]:
!rm -R /content/data/Person1/.ipynb_checkpoints

rm: cannot remove '/content/data/Person1/.ipynb_checkpoints': No such file or directory


In [23]:
optimizer = optim.Adam(resnet.parameters(), lr=0.001)
scheduler = MultiStepLR(optimizer, [5, 10])

trans = transforms.Compose([
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])

dataset = datasets.ImageFolder("/content/data/", transform=trans)
img_inds = np.arange(len(dataset))
np.random.shuffle(img_inds)
train_inds = img_inds[:int(0.8 * len(img_inds))]
val_inds = img_inds[int(0.8 * len(img_inds)):]

train_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(train_inds)
)
val_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(val_inds)
)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [24]:
loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

In [25]:
writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10

print('\n\nInitial')
print('-' * 10)
resnet.eval()
training.pass_epoch(
    resnet, loss_fn, val_loader,
    batch_metrics=metrics, show_running=True, device=device,
    writer=writer
)

for epoch in range(epochs):
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)

    resnet.train()
    training.pass_epoch(
        resnet, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

    resnet.eval()
    training.pass_epoch(
        resnet, loss_fn, val_loader,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

writer.close()



Initial
----------


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Valid |     1/1    | loss:    0.0000 | fps:    0.5092 | acc:    1.0000   

Epoch 1/30
----------
Train |     3/3    | loss:    0.0000 | fps:    2.5868 | acc:    1.0000   
Valid |     1/1    | loss:    0.0000 | fps:    5.1826 | acc:    1.0000   

Epoch 2/30
----------
Train |     3/3    | loss:    0.0000 | fps:    2.6611 | acc:    1.0000   
Valid |     1/1    | loss:    0.0000 | fps:    4.5213 | acc:    1.0000   

Epoch 3/30
----------
Train |     3/3    | loss:    0.0000 | fps:    2.6767 | acc:    1.0000   
Valid |     1/1    | loss:    0.0000 | fps:    4.5198 | acc:    1.0000   

Epoch 4/30
----------
Train |     3/3    | loss:    0.0000 | fps:    2.6315 | acc:    1.0000   
Valid |     1/1    | loss:    0.0000 | fps:    4.4488 | acc:    1.0000   

Epoch 5/30
----------
Train |     3/3    | loss:    0.0000 | fps:    2.6134 | acc:    1.0000   
Valid |     1/1    | loss:    0.0000 | fps:    2.8543 | acc:    1.0000   

Epoch 6/30
----------
Train |     3/3    | loss:    0.0000 | fps:    1

In [26]:
torch.save(resnet.state_dict(), '/content/facenet_model_final.pth')